# Vitessce Widget Tutorial

# Visualization of SpatialData Object

## 1. Import dependencies

We need to import the classes and functions that we will be using from the corresponding packages.

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from urllib.request import urlretrieve
import dask

dask.config.set({'dataframe.query-planning-warning': False})

from spatialdata import read_zarr
import scanpy as sc

from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    AbstractWrapper,
    SpatialDataWrapper
)
from vitessce.data_utils import (
    optimize_adata,
    VAR_CHUNK_SIZE,
)
import zipfile


In [2]:
zip_filepath = Path("data/visium.spatialdata.zarr.zip")
spatialdata_filepath = zip_filepath.with_suffix('')
if not zip_filepath.exists():
    spatialdata_filepath.parent.mkdir(exist_ok=True)
    urlretrieve('https://s3.embl.de/spatialdata/spatialdata-sandbox/visium_associated_xenium_io.zip', zip_filepath)
    with zipfile.ZipFile(zip_filepath,"r") as zip_ref:
        zip_ref.extractall(spatialdata_filepath.parent)
        (spatialdata_filepath.parent / "data.zarr").rename(spatialdata_filepath)
        (spatialdata_filepath / "tables").rename(spatialdata_filepath / "table") # TODO: remove once fixed in Vitessce

## 3. Load the data

Note: this function may print a `FutureWarning`

In [3]:
spatialdata = read_zarr(spatialdata_filepath)

/var/folders/k9/9wc7lvwj2g34_r74kn6cr0nr0000gn/T/ipykernel_98958/1452425863.py:1: DeprecationWarning: Table group found in zarr store at location /Users/ilangold/Projects/Gehlenborg/vitessce-python/docs/notebooks/data/visium.spatialdata.zarr. Please update the zarr storeto use tables instead.
  spatialdata = read_zarr(spatialdata_filepath)


In [4]:
spatialdata

SpatialData object with:
├── Images
│     ├── 'CytAssist_FFPE_Human_Breast_Cancer_full_image': MultiscaleSpatialImage[cyx] (3, 21571, 19505), (3, 10785, 9752), (3, 5392, 4876), (3, 2696, 2438), (3, 1348, 1219)
│     ├── 'CytAssist_FFPE_Human_Breast_Cancer_hires_image': SpatialImage[cyx] (3, 2000, 1809)
│     └── 'CytAssist_FFPE_Human_Breast_Cancer_lowres_image': SpatialImage[cyx] (3, 600, 543)
├── Shapes
│     └── 'CytAssist_FFPE_Human_Breast_Cancer': GeoDataFrame shape: (4992, 2) (2D shapes)
└── Tables
      └── 'table': AnnData (4992, 18085)
with coordinate systems:
▸ 'downscaled_hires', with elements:
        CytAssist_FFPE_Human_Breast_Cancer_hires_image (Images), CytAssist_FFPE_Human_Breast_Cancer (Shapes)
▸ 'downscaled_lowres', with elements:
        CytAssist_FFPE_Human_Breast_Cancer_lowres_image (Images), CytAssist_FFPE_Human_Breast_Cancer (Shapes)
▸ 'global', with elements:
        CytAssist_FFPE_Human_Breast_Cancer_full_image (Images), CytAssist_FFPE_Human_Breast_Cancer (Shap

## 4. Create the Vitessce widget configuration

Vitessce needs to know which pieces of data we are interested in visualizing, the visualization types we would like to use, and how we want to coordinate (or link) the views.

### 4.1. Instantiate a `VitessceConfig` object

Use the `VitessceConfig` constructor to create an instance.

In [5]:
vc = VitessceConfig(schema_version="1.0.16", name='Visium SpatialData Demo (visium_associated_xenium_io)', description='From https://spatialdata.scverse.org/en/latest/tutorials/notebooks/datasets/README.html')

### 4.2. Add a dataset to the `VitessceConfig` instance

In Vitessce, a dataset is a container for one file per data type. The `.add_dataset(name)` method on the `vc` instance sets up and returns a new dataset instance.

Then, we can call the dataset's `.add_object(wrapper_object)` method to attach a "data wrapper" instance to our new dataset. For example, the `AnnDataWrapper` helps to configure AnnData Zarr stores for use in the Vitessce configuration.

Dataset wrapper classes may require additional parameters to resolve ambiguities. For instance, `AnnData` objects may store multiple clusterings or cell type annotation columns in the `adata.obs` dataframe. We can use the parameter `obs_set_paths` to tell Vitessce that certain columns of the `obs` dataframe correspond to cell type annotations or cell clusterings.

In [6]:
wrapper = SpatialDataWrapper(
    spatialdata_path=spatialdata_filepath.absolute(),
    image_path="images/CytAssist_FFPE_Human_Breast_Cancer_full_image",
    obs_feature_matrix_path = "table/table/X",
    feature_labels_path = "table/table/var/gene_ids",
    shapes_path = "shapes/CytAssist_FFPE_Human_Breast_Cancer"
)

In [7]:
dataset = vc.add_dataset(name='Breast Cancer Visium').add_object(wrapper)

In [8]:
spatial = vc.add_view("spatialBeta", dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)

### 4.4. Define the visualization layout

The `vc.layout(view_concat)` method allows us to specify how our views will be arranged in the layout grid in the widget. The `|` and `/` characters are magic syntax for `hconcat(v1, v2)` and `vconcat(v1, v2)`, respectively.

In [9]:
vc.layout(spatial | genes);

In [15]:
vw = vc.widget()
vw

VitessceWidget(config={'version': '1.0.16', 'name': 'Visium SpatialData Demo (visium_associated_xenium_io)', '…

In [14]:
vw.close()

In [11]:
vc.to_dict('')

{'version': '1.0.16',
 'name': 'Visium SpatialData Demo (visium_associated_xenium_io)',
 'description': 'From https://spatialdata.scverse.org/en/latest/tutorials/notebooks/datasets/README.html',
 'datasets': [{'uid': 'A',
   'name': 'Breast Cancer Visium',
   'files': [{'fileType': 'spatialdata.zarr',
     'url': '/A/0/94f0b436-6c22-4adf-ba35-ce35125b467c.spatialdata.zarr',
     'options': {'obsFeatureMatrix': {'path': 'table/table/X'},
      'obsSpots': {'path': 'shapes/CytAssist_FFPE_Human_Breast_Cancer',
       'tablePath': 'table/table',
       'region': 'region'},
      'image': {'path': 'images/CytAssist_FFPE_Human_Breast_Cancer_full_image'},
      'featureLabels': {'path': 'table/table/var/gene_ids'}}}]}],
 'coordinationSpace': {'dataset': {'A': 'A'}},
 'layout': [{'component': 'spatialBeta',
   'coordinationScopes': {'dataset': 'A'},
   'x': 0.0,
   'y': 0,
   'w': 6.0,
   'h': 12},
  {'component': 'featureList',
   'coordinationScopes': {'dataset': 'A'},
   'x': 6.0,
   'y': 0